## Understand the datasets

To complete the portfolio project in this Jupyter Notebook, I used three datasets that are available on the city of Chicago's Data Portal:


### 1. Socioeconomic Indicators in Chicago

This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
[https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2]

### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
[https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t]

### 3. Chicago Crime Data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:
[https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2]


### Modify the datasets

This portfolio project requires me to have these three tables populated with a subset of the whole datasets.


**NOTE:** The datasets used are subsets of the original datasets and have some of the column names modified to be more database friendly which will make it easier to complete this project.


### Store the datasets in database tables

To analyze the data using SQL, it first needs to be stored in the database. I used IBM Db2 database on IBM Cloud for this project.

It is highly recommended to manually load the table using the database console LOAD tool. Click on create "(+) New Table" and specify the name of the table to create and then click "Next".

Open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the first dataset, Next create a New Table to load the data. 

I named the new tables as follows:

1.  **CENSUS_DATA**
2.  **CHICAGO_PUBLIC_SCHOOLS**
3.  **CHICAGO_CRIME_DATA**


### Connect to the database

First, I loaded the SQL extension and established a connection with the database

I installed these libraries by removing the `#` sign before `!pip` in the code cell below.


In [ ]:
# Uncomment lines below to install them:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [1]:
%load_ext sql

In the next cell, I entered db2 connection string.

In [2]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
# Enter the connection string for your Db2 on Cloud database instance below
%sql ibm_db_sa://username:password@hostname:port/db-name?security=SSL

'Connected: kfc91342@bludb'

## Problems

I wrote and executed SQL queries to solve problems relating to the datasets

### Problem 1

##### The total number of crimes recorded in the CRIME table:


In [ ]:
%%sql
SELECT COUNT(*)AS NUMBER_OF_CRIMES 
FROM CHICAGO_CRIME_DATA;

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


number_of_crimes
533


### Problem 2

##### Community areas with per capita income less than 11000:


In [23]:
%%sql
SELECT COMMUNITY_AREA_NAME, PER_CAPITA_INCOME 
FROM CENSUS_DATA 
WHERE PER_CAPITA_INCOME < 11000;

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


community_area_name,per_capita_income
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


### Problem 3

##### All case numbers for crimes involving minors? (children are not considered minors for the purposes of crime analysis):


In [24]:
%%sql 
SELECT CASE_NUMBER
FROM CHICAGO_CRIME_DATA 
WHERE DESCRIPTION LIKE '%MINOR%';

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


case_number
HL266884
HK238408


### Problem 4

##### All kidnapping crimes involving a child:


In [25]:
%%sql 
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'; 

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


case_number,primary_type,description
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


### Problem 5

##### Type of crimes were recorded at schools:


In [26]:
%%sql
SELECT DISTINCT(PRIMARY_TYPE)
FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


primary_type
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPA
NARCOTICS
PUBLIC PEACE VI


### Problem 6

##### The average safety score for each type of school:


In [35]:
%%sql
SELECT "Elementary, Middle, or High School",AVG(SAFETY_SCORE) AS AVERAGE_SAFETY_SCORE
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School";

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


"Elementary, Middle, or High School",average_safety_score
ES,49
HS,49
MS,48


### Problem 7

##### 5 community areas with highest % of households below poverty line:


In [28]:
%%sql
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


community_area_name,percent_households_below_poverty
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8

##### Community area that is most crime prone:


In [32]:
%%sql
SELECT CCD.COMMUNITY_AREA_NUMBER, COUNT(CCD.COMMUNITY_AREA_NUMBER) AS FREQUENCY
FROM CHICAGO_CRIME_DATA AS CCD 
GROUP BY CCD.COMMUNITY_AREA_NUMBER
ORDER BY COUNT(CCD.COMMUNITY_AREA_NUMBER) DESC
LIMIT 1

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


community_area_number,frequency
25,43


### Problem 9

#####  The name of the community area with highest hardship index:


In [33]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM  CENSUS_DATA
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


community_area_name
Riverdale


### Problem 10

##### The Community Area Name with most number of crimes:


In [34]:
%%sql
SELECT community_area_name
FROM CENSUS_DATA 
WHERE COMMUNITY_AREA_NUMBER = (    
    SELECT CCD.COMMUNITY_AREA_NUMBER 
    FROM CHICAGO_CRIME_DATA AS CCD 
    GROUP BY CCD.COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(CCD.COMMUNITY_AREA_NUMBER) DESC
    LIMIT 1)

LIMIT 1;

 * ibm_db_sa://kfc91342:***@21fecfd8-47b7-4937-840d-d791d0218660.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31864/bludb
Done.


community_area_name
Austin
